In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Review_Analysis").getOrCreate()

In [30]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
#df.describe()
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

In [48]:
# Filter data where total votes is equal or greater than 20
total_votes_df = vine_df.filter(vine_df["total_votes"] >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [49]:
# Check data types
#total_votes_df.describe()

# convert data types
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

# Convert data types of int columns
total_votes_df.withColumn("helpful_votes",col("helpful_votes").cast("int"))
total_votes_df.withColumn("total_votes",col("total_votes").cast("int"))
total_votes_df.withColumn("star_rating",col("star_rating").cast("int"))


DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [68]:
# filter where no of helpful_votes divided by no of total_votes is equal to or greater than 50%

helpful_votes_df = total_votes_df.filter(((total_votes_df["helpful_votes"] / total_votes_df["total_votes"])*100) >= 50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [90]:
# filter for vine-sponsored reviews

vine_yes_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'Y')
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
|R2T83WIPG3IX5D|          4|           72|         76|   Y|                N|
| RD006IWFCUA5A|          5|          122|        133|   Y|                N|
| REOQCBUVUPXP1|          4|           23|         27|   Y|                N|
|R2336R80ZEOKTX|          5|           63|         63|   Y|                N|
| RC1ZXE5GVL9TA|          4|           20|         23|   Y|     

In [91]:
# filter for none vine-sponsored reviews

vine_no_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'N')
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [92]:
# vine_yes_df analysis

# total number of reviews
total_reviews_yes = vine_yes_df.count()

# total number of 5 star reviews
total_five_yes = vine_yes_df.filter(vine_yes_df["star_rating"] == 5).count()

# share of 5 star reviews
share_five_yes = round(((total_five_yes / total_reviews_yes)*100), 2)

print(f"There are {total_reviews_yes} total reviews that were paid for by the Vine program")
print(f"Of those, {total_five_yes} are five-star reviews")
print(f"{share_five_yes}% of all paid Vine reviews are five-star reviews")

There are 21 total reviews that were paid for by the Vine program
Of those, 10 are five-star reviews
47.62% of all paid Vine reviews are five-star reviews


In [93]:
# vine_no_df analysis

# total number of reviews
total_reviews_no = vine_no_df.count()

# total number of 5 star reviews
total_five_no = vine_no_df.filter(vine_no_df["star_rating"] == 5).count()

# share of 5 star reviewsshare_five_no = round(((total_five_no / total_reviews_no)*100), 2)

print(f"There are {total_reviews_no} total reviews that were not paid for by the Vine program")
print(f"Of those, {total_five_no} are five-star reviews")
print(f"{share_five_no}% of all non-paid Vine reviews are five-star reviews")

There are 6690 total reviews that were not paid for by the Vine program
Of those, 3448 are five-star reviews
51.54% of all non-paid Vine reviews are five-star reviews
